In [1]:
import pickle
import pandas as pd

In [2]:
from google.colab import drive

drive.mount('/content/gdrive/')
% cd '/content/gdrive/MyDrive/Colab Notebooks/university'

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Colab Notebooks/university


In [130]:
with open('confs.pkl', 'rb') as file:
  pred_confs = pickle.load(file)

dev_pred_asp = pd.read_csv(
    'dev_pred_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)

In [131]:
dev_pred_asp['conf'] = pred_confs

In [132]:
mapper = {'positive': 1, 'neutral': 0, 'negative': -1}

In [133]:
dev_pred_asp['grade'] = dev_pred_asp.sentiment.map(mapper)

Нормализуем уверенности в пределах одной категории для каждого текста

In [134]:
dev_pred_asp['conf'] = dev_pred_asp['conf'] / dev_pred_asp.groupby(['text_id', 'category'], observed=True)['conf'].transform('sum')

In [135]:
dev_pred_asp.head()

,text_id,category,mention,start,end,sentiment,conf,grade
0,13823,Service,приняла заказ,254,267,positive,0.083149,1
1,13823,Service,дала меню,208,217,positive,0.083149,1
2,13823,Service,качество обслуживания,671,692,positive,0.125830,1
3,13823,Service,проводила к столу,188,205,positive,0.083149,1
4,13823,Whole,ресторан,337,345,positive,0.142316,1


Посчитаем взвешенную сумму оценок для каждой категории по каждому тексту (вес - нормализованная уверенность модели)

In [136]:
dev_pred_asp['weigh_grade'] = dev_pred_asp.grade * dev_pred_asp.conf

In [137]:
aggregated = pd.DataFrame(dev_pred_asp.groupby(['text_id', 'category']).agg({'weigh_grade':['count', 'sum']}))

In [138]:
aggregated.head()

weigh_grade          
                       count       sum
text_id category                      
785     Food               5 -0.000927
        Interior           3  0.758322
        Price              1  0.000000
        Service            7  0.607027
        Whole              3  1.000000

Поделим распределение получившихся оценок на три равных отрезка

In [139]:
def get_cat_label(weigh_grade, count):
  if count == 0:
    return 'absence'
  elif weigh_grade > 0.33:
    return 'positive'
  elif weigh_grade < -0.33:
    return 'negative'
  else:
    return 'neutral'


aggregated['result'] = aggregated.apply(lambda x: get_cat_label(x[('weigh_grade', 'sum')], x[('weigh_grade', 'count')]), axis=1)

In [140]:
results = aggregated.result.to_dict()

In [141]:
text_ids = list(set([ind[0] for ind in aggregated.index]))

In [142]:
categories = ['Food', 'Interior', 'Price', 'Service', 'Whole']

In [143]:
with open('dev_pred_cats.txt', 'w') as f:
  for text_id in  text_ids:
    for cat in categories:
      if (text_id, cat) in results:
        print(text_id, cat, results[(text_id, cat)], sep="\t", file=f)
      else:
        print(text_id, cat, 'absence', sep="\t", file=f)